In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


######################################################
# General
######################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from time import sleep
import time
from multiprocessing import Pool


######################################################
# Music Stuff
######################################################
from dbBase import dbBase
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData


######################################################
# Versions
######################################################
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-11-09 18:43:56.280704


# Initialize

In [2]:
%load_ext autoreload
%autoreload
discLM = discogs('lastfm')
dutils = lastfmUtils()
artLM  = artistLM()
artsLM = artistsLM(discLM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


NameError: name 'discogs' is not defined

# Download Known Artists

In [ ]:
def getMyMusicDBMap():
    myMusicMap = getFile(ifile="/Users/tgadfort/Documents/code/music/myMusicMap.p")
    print("Found {0} entries".format(len(myMusicMap)))
    return myMusicMap

myMusicMap = getMyMusicDBMap()

In [ ]:
def downloadSearchResults(maxI=-1):
    for i,artistName in enumerate(myMusicMap.keys()):
        if i <= maxI:
            continue
        print("====>",i,"/",len(myMusicMap),"  : ",artistName)
#        try:
        artsLM.searchLastFMForArtist(artistName, sleeptime=3)
#        except:
#            sleep(3)
        sleep(3)

In [ ]:
downloadSearchResults(3544)

# Download Extra Artist Files

In [ ]:
#artsLM.assertDBModValExtraData(1, maxPages=3)

In [5]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata
pdb = parseDBArtistsData(maindb=maindb)

Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4739 artist entries
Counter({'Discogs': 4739, 'AllMusic': 4739, 'MusicBrainz': 4739, 'AceBootlegs': 4739, 'RateYourMusic': 4739, 'LastFM': 4739, 'DatPiff': 4739, 'RockCorner': 4739, 'CDandLP': 4739, 'MusicStack': 4739, 'MetalStorm': 4739})
Counter({'Discogs': 4739, 'AllMusic': 4739, 'MusicBrainz': 4739, 'AceBootlegs': 4739, 'RateYourMusic': 4739, 'LastFM': 4739, 'DatPiff': 4739, 'RockCorner': 4739, 'CDandLP': 4739, 'MusicStack': 4739, 'MetalStorm': 4739})
  MusicDB is ok
  Reverse MusicDB is ok
Setting Basic Database Objects
=========================   discogs   =========================
Directory Information:
  Raw:  /Volumes/Piggy/Discog
  Meta: /Users/tgadfort/Music/Discog
  DB:   /Users/tgadfort/Music/Discog
==

In [ ]:
from random import shuffle
modVals = list(range(100))
shuffle(modVals)
for modVal in modVals:
    artsLM.assertDBModValExtraData(modVal, maxPages=25)
    continue
    #except:
    #    continue

In [ ]:
artsLM.assertDBModValExtraData(61)

In [ ]:
retval = artLM.getData("/Volumes/Biggy/Discog/artists-lastfm/51/34806831451.p")

In [ ]:
artsLM.parseArtistFiles()

In [ ]:
dutils.getArtistID("Coldplay vs. The Killers vs. Kelly Clarkson vs. Bon Jovi vs. Empire Of The Sun vs. Muse")  # 99596595167
dutils.getArtistID("Coldplay vs. The Killers vs. Kelly Clarkson vs. Bon…")   # 93496498096

In [ ]:
url="https://www.last.fm/search/artists?q=Coldplay"
savename="lm_search.p"
artsLM.downloadArtistURL(url=url, savename=savename)

In [ ]:
bsdata = getHTML("lm_search.p")

In [ ]:
uls = bsdata.findAll("ul", {"class": "artist-results"})
for ul in uls:
    lis = ul.findAll("li", {"class": "artist-result"})
    for li in lis:
        h4 = li.find("h4")
        if h4 is None:
            raise ValueError("No h4 in list")
        result   = artLM.getNamesAndURLs(h4)
        name     = result[0].name
        url      = result[0].url
        artistID = dutils.getArtistID(name)
        print(name,'\t',url,'\t',artistID)

In [ ]:
#bsdata

In [ ]:
retval = artLM.getData("lastfm.p")

In [ ]:
retval.artist.get()

In [ ]:
retval.pages.get()

In [ ]:
retval.url.get()

In [ ]:
retval.ID.get()

In [ ]:
retval.profile.get()

In [ ]:
for album in retval.media.media["Albums"]:
    print(album.get())

In [ ]:
retval.mediaCounts.get()

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs('lastfm')
arts = artists(disc)
#arts.searchDiscogForArtist("The Whispers")

In [ ]:
from webUtils import getWebData
url="https://www.last.fm/music/Coldplay/+albums"
url="https://www.last.fm/music/Coldplay/+albums?order=release_date"
url="https://www.last.fm/music/Coldplay/+albums?order=release_date"
savename="lastfm.p"
getWebData(base=url, savename=savename, useSafari=False)

In [ ]:
url="https://www.datpiff.com/mixtapes-search?criteria=yo%20gotti&sort=relevance"
savename = "dp_search.p"
getWebData(base=url, savename=savename, useSafari=False)

In [ ]:
getWebData

In [ ]:
bsdata = getHTML("dp_search.p")

In [ ]:
bsdata

In [ ]:
contentdivs = bsdata.findAll("div", {"class": "contentItem"})
for i,contentdiv in enumerate(contentdivs):
    artistDiv = contentdiv.find("div", {"class": "artist"})
    if artistDiv is None:
        raise ValueError("There is no artist div")
    artistName = artistDiv.text
    
    albumDiv = contentdiv.find("div", {"class": "title"})
    if albumDiv is None:
        raise ValueError("There is no title div")
    albumName = albumDiv.text
    try:
        albumURL  = albumDiv.find("a").attrs['href']
    except:
        albumURL  = None
        
    print(artistName,'\t',albumName)

In [ ]:
divs[1]